In [29]:
import json
from tld import get_fld


In [30]:
page_load_time_desktop=[]
page_load_time_mobile=[]
num_requests_desktop=[]
num_requests_mobile=[]
num_third_parties_desktop=[]
num_third_parties_mobile=[]
num_trackers_domain_desktop=[]
num_trackers_domain_mobile=[]
num_trackers_ent_desktop=[]
num_trackers_ent_mobile=[]




In [53]:
#load the disconnect blocklist as top level domains
f = open('services.json') 
disconnect = json.load(f) 
disconnect=disconnect['categories']
tracker_domain_list=set()
for entry in disconnect:
   temp=disconnect[entry]
   keys=temp.keys() 

SyntaxError: unexpected EOF while parsing (<ipython-input-53-14ddb37b5118>, line 8)

In [31]:
def get_page_load_time(website):
    start_time=website['pageload_start_ts']
    end_time=website['pageload_end_ts']
    return end_time-start_time

In [32]:
def get_num_requests(website):
    return len(website['requests'])


In [33]:
def get_third_parties(website):
    third_party_domains=set()
    first_domain=website['domain']
    for req in website['requests']:
        req_domain=get_fld(req['request_url'])
        if (req_domain!=first_domain):
            third_party_domains.add(req_domain)
    return third_party_domains

In [ ]:
def get_tracker_domain():

In [34]:
f = open('facebook.com.json') 
data = json.load(f) 


In [35]:
print (get_page_load_time(data))
print (get_num_requests(data))
print(get_third_parties(data))

1.3114988803863525
59
4
